In [4]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras_vggface.vggface import VGGFace
from keras_vggface.utils import preprocess_input as preprocess_input_vggface
import matplotlib.pyplot as plt
from livelossplot.tf_keras import PlotLossesCallback

In [ ]:
# Загрузка данных
df = pd.read_csv('./_datasets/train.csv', index_col=0)
df['image_path'] = df['image_path'].apply(lambda x: x.replace('train', '_datasets/train_dataset'))

# Разделение данных на тренировочные и тестовые выборки
train_df = df.sample(frac=0.8, random_state=200)
test_df = df.drop(train_df.index)

def preprocess_input_facenet(image_):
    """
    image_ -- тензор размера (1, H, W, 3)
    return: картинка, с примененным preprocess_input(..., version=2) из keras_vggface
    """
    preprocessed = preprocess_input_vggface(image_, version=2)
    return preprocessed

BATCH_SIZE = 16
IMAGE_SIZE = 224
N_CLASSES = 9

# Генератор для тренировочных данных с аугментацией
train_generator = ImageDataGenerator(
    rotation_range=15,
    horizontal_flip=True,
    fill_mode='nearest',
    brightness_range=(0.5, 1.5),
    shear_range=0.1,
    zoom_range=0.1,
    preprocessing_function=preprocess_input_facenet
)

train_data_gen = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='image_path',
    y_col='emotion',
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='sparse',
    shuffle=True,
    interpolation='nearest'
)

# Генератор для тестовых данных без аугментации
val_generator = ImageDataGenerator(preprocessing_function=preprocess_input_facenet)

val_data_gen = val_generator.flow_from_dataframe(
    dataframe=test_df,
    x_col='image_path',
    y_col='emotion',
    target_size=(IMAGE_SIZE, IMAGE_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='sparse',
    shuffle=False,
    interpolation='nearest'
)

# Загрузка модели VGGFace с использованием ResNet50
vggface_model = VGGFace(model='resnet50', include_top=False, input_shape=(224, 224, 3))
vggface_model.summary()

# Добавление дополнительных слоев к модели
model = Sequential([
    vggface_model,
    keras.layers.GlobalAveragePooling2D(),
    keras.layers.Dense(512, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(256, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(N_CLASSES, activation='softmax')
])

# Настройка компиляции модели
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=5e-4,
    decay_steps=100,
    decay_rate=0.9
)

model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.Adam(learning_rate=lr_schedule),
    metrics=[tf.keras.metrics.SparseCategoricalAccuracy()]
)

# Определение коллбеков для сохранения модели и изменения скорости обучения
checkpoint = ModelCheckpoint(
    './_models/_checkpoints/classification/best_model.h5',
    verbose=1,
    save_best_only=True,
    save_weights_only=True,
    save_freq='epoch',
    mode='min'
)

# Обучение модели
EPOCHS = 50
history = model.fit(
    train_data_gen, 
    epochs=EPOCHS, 
    validation_data=val_data_gen, 
    callbacks=[checkpoint, PlotLossesCallback()]
)

# Оценка модели
model.evaluate(val_data_gen)

# Визуализация результатов
def deprocess_image(vggface_image):
    image = np.copy(vggface_image)
    image[..., 0] += 91.4953
    image[..., 1] += 103.8827
    image[..., 2] += 131.0912
    image = image[..., ::-1]
    image = image.astype(np.uint8)
    return image

def show_faces(images, real_emotion=None, predicted_emotion=None):
    plt.figure(figsize=(10, 10))
    emotion_mapping = dict(list(enumerate(('Anger', 'Contempt', 'Disgust', 'Fear', 'Happy', 'Neutral', 'Sad', 'Surprise', 'Uncertain'))))
    for i in range(16):
        plt.subplot(4, 4, i+1)
        plt.xticks([])
        plt.yticks([])
        plt.grid(False)
        plt.imshow(deprocess_image(images[i]))
        real_str = f"Real: {emotion_mapping[int(real_emotion[i])]}, " if real_emotion is not None else "Real: N/A, "
        pred_str = f"Pred: {emotion_mapping[np.argmax(predicted_emotion[i])]}, " if predicted_emotion is not None else "Pred: N/A, "
        correct = (real_emotion is not None and np.argmax(predicted_emotion[i]) == int(real_emotion[i]))
        title_obj = plt.title(f"{real_str}\n{pred_str}")
        plt.subplots_adjust(wspace=0.4)
        if not correct:
            plt.setp(title_obj, color='r')

sample_validation_images, sample_validation_labels = next(val_data_gen)
predicted = model.predict(sample_validation_images)
show_faces(sample_validation_images, real_emotion=sample_validation_labels, predicted_emotion=predicted)

# Сохранение модели и весов
model_json = model.to_json()
with open("./_models/emotion_classification/model.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights('./_models/emotion_classification/saved_weights.h5')